In [22]:
import girder_client
import pandas as pd
import numpy as np
import json

In [2]:
gc = girder_client.GirderClient(apiUrl='https://isic-archive.com/api/v1')
gc.authenticate('dominikkiefer', 'dlproject')

In [3]:
pd.DataFrame(gc.listCollection())

,_accessLevel,_id,_modelType,created,description,name,public,size,updated
0,-1,55943cff9fc3c13155bcad61,collection,2015-07-01T19:18:23.988000+00:00,Clinical feature annotation studies,Annotation Studies,True,0,2016-08-24T03:06:20.532000+00:00
1,-1,55943cff9fc3c13155bcad5e,collection,2014-09-29T02:59:17.932000+00:00,All public lesion image datasets,Lesion Images,True,293175840740,2016-08-24T03:07:21.780000+00:00


In [4]:
#gc.downloadResource("55943cff9fc3c13155bcad5e",dest="/home/dwx/tmp")

dataset_df = pd.DataFrame(gc.get("/dataset"))
dataset_df

,_id,name,updated
0,5627f42b9fc3c132be08d84f,ISIC_MSK-1_1,2015-10-21T20:33:21.466000+00:00
1,582b8d419fc3c1566bbcb098,ISIC_MSK-1_2,2017-04-13T20:08:46.159000+00:00
2,5627f5f69fc3c132be08d852,ISIC_MSK-2_1,2015-10-21T20:33:29.916000+00:00
3,57eebe389fc3c12a89bb75f7,ISIC_MSK-3_1,2017-04-13T20:07:38.035000+00:00
4,581cd6059fc3c13dcd0e0930,ISIC_MSK-4_1,2017-04-13T20:08:05.005000+00:00
5,5825fd959fc3c171066d3352,ISIC_MSK-5_1,2017-04-13T20:08:24.725000+00:00
6,5627eefe9fc3c132be08d84c,ISIC_SONIC_1,2016-08-10T15:36:43.866000+00:00
7,54b6e869bae4785ee2be8652,ISIC_UDA-1_1,2014-11-10T02:39:56.492000+00:00
8,54ea816fbae47871b5e00c80,ISIC_UDA-2_1,2015-03-06T14:54:30.984000+00:00


In [5]:
cur_dataset_id = dataset_df[dataset_df["name"] == "ISIC_MSK-2_1"]._id
cur_dataset_id = cur_dataset_id.tolist()[0]
cur_dataset_id

'5627f5f69fc3c132be08d852'

In [6]:
path = "/dataset/"+str("5627f5f69fc3c132be08d852")+""
gc.get(path)

{'_id': '5627f5f69fc3c132be08d852',
 '_modelType': 'dataset',
 'anonymous': True,
 'created': '2015-06-26T14:24:02.672000+00:00',
 'creator': {'_id': '54cb974fbae47819d8e4c727', 'name': 'User GQZH'},
 'description': 'Benign and malignant skin lesions.\nBiopsy-confirmed melanocytic and non-melanocytic lesions.',
 'license': 'CC-0',
 'metadataFiles': [],
 'name': 'ISIC_MSK-2_1',
 'owner': 'Memorial Sloan Kettering Cancer Center',
 'updated': '2015-10-21T20:33:29.916000+00:00'}

#### Get Image Metadata

In [50]:
path = "/image/"

curSelection = gc.get(path,parameters={"datasetId":cur_dataset_id, "limit":5})

In [51]:
len(curSelection)

5

In [57]:
def getSpecificMetadata(jsonData):
    #result += [jsonData["meta"]["clinical"]["benign_malignant"]]
    result = jsonData["meta"]["clinical"]
    result["name"] = jsonData["name"]
    return result
    
#dir(curSelection)
path = "/image/"
counter = 0
malignant_counter = 0
result = []
for a in curSelection:
    counter += 1
    curId = a["_id"]
    curMetadata = gc.get(path+str(curId),parameters={})
    specificMetadata = getSpecificMetadata(curMetadata)
    #if specificMetadata[1] == "malignant":
    #    malignant_counter +=1
    result += [specificMetadata]
    if counter % 60 == 0:
        print("cur ratio malignant/total: ",malignant_counter/counter)

In [59]:
pd.DataFrame(result)

,age_approx,benign_malignant,clin_size_long_diam_mm,diagnosis,diagnosis_confirm_type,family_hx_mm,melanocytic,name,personal_hx_mm,sex
0,70,malignant,4.0,melanoma,histopathology,False,True,ISIC_0009868,False,female
1,40,benign,4.0,nevus,histopathology,False,True,ISIC_0009869,True,female
2,40,benign,3.0,nevus,histopathology,False,True,ISIC_0009870,True,female
3,45,benign,5.0,nevus,histopathology,False,True,ISIC_0009871,False,male
4,50,benign,3.0,nevus,histopathology,False,True,ISIC_0009872,False,female
